### 타깃
[naver news:공공자전거](https://search.naver.com/search.naver?&where=news&query=%EA%B3%B5%EA%B3%B5%EC%9E%90%EC%A0%84%EA%B1%B0&sm=tab_pge&sort=0&photo=0&field=0&reporter_article=&pd=0&ds=&de=&docid=&nso=so:r,p:all,a:all&mynews=0&start=1&refresh_start=0)

#### Ref
- [selenium attribute](https://stackoverflow.com/questions/30324760/how-to-get-attribute-of-element-from-selenium?utm_medium=organic&utm_source=google_rich_qa&utm_campaign=google_rich_qa)
- [~~selenium multiple class~~](https://stackoverflow.com/questions/21713280/find-div-element-by-multiple-class-names?utm_medium=organic&utm_source=google_rich_qa&utm_campaign=google_rich_qa) selenium옛버전. 이젠 contains 나 findElement는 안됨

In [86]:
# coding: utf-8
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

import os.path
from os import rename
from os import listdir

import time

import csv

In [109]:
def startWeb(numPage=1):
    """
    @return driver
    """
    driver= webdriver.Chrome('lib/chromedriver.exe')
    paging= str(1+10*(numPage-1))
    # caution: "\ is before &"
    url= "https://search.naver.com/search.naver?&where=news\
    &query=%EA%B3%B5%EA%B3%B5%EC%9E%90%EC%A0%84%EA%B1%B0&sm=tab_pge\
    &sort=0&photo=0&field=0&reporter_article=&pd=0&ds=&de=&docid=\
    &nso=so:r,p:all,a:all&mynews=1&start="+paging+"&refresh_start=0"
    driver.get(url)
    
    try:
        element= WebDriverWait(driver, 10).until(
            # is comming content?
            EC.presence_of_element_located((By.XPATH, '//li'))
        )
    except Exception as e:
        print('error',e)
    return driver

#### 언론사 선택

In [88]:
def searchOptPress():
    """
    일간지, 뉴시스, 연합뉴스, 한국경제TV, JTBC, 시사IN
    """
    #검색옵션
    xpath1= '//a[@id="_search_option_btn"]'
    element= driver.find_element_by_xpath(xpath1)
    element.click()

    #언론사
    xpath1= '//li[@id="news_popup"]'
    element= driver.find_element_by_xpath(xpath1)
    element.click()

    #전체선택 해제
    xpath2= '//input[@id="select_all_order"]'
    element= driver.find_element_by_xpath(xpath2)
    element.click()

    #일간지 선택
    xpath3= '//input[@id="ca_p1"]'
    element= driver.find_element_by_xpath(xpath3)
    element.click()
    
    somePress= ["뉴시스", "연합뉴스", "한국경제TV", "JTBC", "시사IN"]
    for press in somePress:
        xpath= '//label[@title="'+press+'"]'
        element= driver.find_element_by_xpath(xpath)
        element.click()
        
    # 확인
    xpathChk= '//button[@class="impact _submit_btn"]'
    element= driver.find_element_by_xpath(xpathChk)
    element.click()
    
    try:
        element= WebDriverWait(driver, 10).until(
            # is comming content?
            EC.presence_of_element_located((By.XPATH, '//li'))
        )
    except Exception as e:
        print('error',e)

#### 링크 얻기

In [110]:
def movePage(numPage):
    paging= str(1+10*(numPage-1))
    url= "https://search.naver.com/search.naver?&where=news\
    &query=%EA%B3%B5%EA%B3%B5%EC%9E%90%EC%A0%84%EA%B1%B0&sm=tab_pge\
    &sort=0&photo=0&field=0&reporter_article=&pd=0&ds=&de=&docid=\
    &nso=so:r,p:all,a:all&mynews=1&start="+paging+"&refresh_start=0"
    driver.get(url)
    
    try:
        element= WebDriverWait(driver, 10).until(
            # is comming content?
            EC.presence_of_element_located((By.XPATH, '//div[@class="thumb"]'))
        )
    except Exception as e:
        print('error',e)

In [154]:
def saveLink(listLink):
    #caution: not '_sp_each_title' but '  _sp_each_title'
    #xpathLink= '//a[@class="_sp_each_url _sp_each_title"]'
    #xpathLink= '//div[contains(@class, "_sp_each_url") and contains(@class, " _sp_each_title")]'
    
    xpathLink1= '//a[@class=" _sp_each_title"]'
    elements= driver.find_elements_by_xpath(xpathLink1)
    for element in elements:
        listLink.append(element.get_attribute("href"))
        
    xpathLink2= '//a[@class="_sp_each_url _sp_each_title"]'
    elements= driver.find_elements_by_xpath(xpathLink2)
    for element in elements:
        listLink.append(element.get_attribute("href"))
        
    return listLink

여기까지 작성

---

#### 각 링크의 내용

In [156]:
def movePageLink(link):
    driver.get(link)
    try:
        element= WebDriverWait(driver, 10).until(
            # is coming a content?
            EC.presence_of_element_located((By.XPATH, '//div'))
        )
    except Exception as e:
        print('error',e)

In [68]:
def getContent():
    """
    @return conTitleText, conDateText, conBodyText, 
    """
    try:
        conTitle= WebDriverWait(driver, 3).until(
            EC.presence_of_element_located((By.XPATH, '//div[@class="board_view01"]/dl/dt'))
        )
        
        conDate= WebDriverWait(driver, 3).until(
            EC.presence_of_element_located((By.XPATH, '//span[@class="date"]'))
        )
        
        conBody= WebDriverWait(driver, 3).until(
            EC.presence_of_element_located((By.XPATH, '//div[@class="view_cont"]'))
        )
        
        conWriter= WebDriverWait(driver, 3).until(
            EC.presence_of_element_located((By.XPATH,  '//p[@class="btm_id"]'))
        )
        
        conTitleText= rmSpecialChr(conTitle.text)
        conDateText= rmSpecialChr(conDate.text)
        conBodyText= rmSpecialChr(conBody.text)
        conWriterText= rmSpecialChr(conWriter.text)
    except Exception as e:
        print('error',e)
        
    return conTitleText, conDateText, conBodyText, conWriterText

In [98]:
def rmSpecialChr(weirdStr):
    """
    @param string
    @return string. #특문제거, \n -> ___
    """
    regPattLine= '\n'
    res= re.sub(regPattLine,'___',weirdStr)
    regPatt= '[^ㄱ-ㅎㅏ-ㅣ가-힣a-zA-Z0-9()": ]|[[\u3131-\u318E\uAC00-\uD7A3]]'
    res= re.sub(regPatt,'',res)
    res= re.sub('\\s+', ' ', res) # 공백 제거 해,말아? 하자. 안 하면 헬일듯
    return res

###### 세션 종료, 자원반납

In [149]:
driver.close()
driver.quit()

### 실행_한줄test

In [150]:
# WELL-DONE !
driver= startWeb()

In [151]:
searchOptPress()

In [152]:
numPage= 1
movePage(numPage)

In [155]:
listLink= list([])
listLink= saveLink(listLink)

listLink

['http://www.newsis.com/view/?id=NISX20180421_0000288016&cID=10803&pID=10800',
 'http://www.seoul.co.kr/news/newsView.php?id=20180417014003&wlog_tag3=naver',
 'http://www.hankookilbo.com/v/0e2f312b60b84e099b87d04436fc3308',
 'http://bizn.donga.com/realestate/3/all/20180416/89637182/2',
 'http://news.joins.com/article/olink/22127647',
 'http://news.joins.com/article/olink/22120203',
 'http://news.chosun.com/site/data/html_dir/2018/04/10/2018041000149.html',
 'http://app.yonhapnews.co.kr/YNA/Basic/SNS/r.aspx?c=AKR20180408025600004&did=1195m',
 'http://www.asiatoday.co.kr/view.php?key=20180422010012584',
 'http://www.asiatoday.co.kr/view.php?key=20180418010010605']

## 실행- 링크얻기_완료

In [158]:
driver= startWeb()
searchOptPress()

In [159]:
numPages= range(1,400)
listLink= list([])

for numPage in numPages:
    print(numPage)
    movePage(numPage)
    listLink= saveLink(listLink)

listLink

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


['http://www.newsis.com/view/?id=NISX20180421_0000288016&cID=10803&pID=10800',
 'http://www.seoul.co.kr/news/newsView.php?id=20180417014003&wlog_tag3=naver',
 'http://www.hankookilbo.com/v/0e2f312b60b84e099b87d04436fc3308',
 'http://bizn.donga.com/realestate/3/all/20180416/89637182/2',
 'http://news.joins.com/article/olink/22127647',
 'http://news.joins.com/article/olink/22120203',
 'http://news.chosun.com/site/data/html_dir/2018/04/10/2018041000149.html',
 'http://app.yonhapnews.co.kr/YNA/Basic/SNS/r.aspx?c=AKR20180408025600004&did=1195m',
 'http://www.asiatoday.co.kr/view.php?key=20180422010012584',
 'http://www.asiatoday.co.kr/view.php?key=20180418010010605',
 'http://www.segye.com/content/html/2018/04/04/20180404005643.html?OutUrl=naver',
 'http://www.newsis.com/view/?id=NISX20180404_0000272095&cID=10801&pID=10800',
 'http://www.hani.co.kr/arti/economy/it/839062.html',
 'http://app.yonhapnews.co.kr/YNA/Basic/SNS/r.aspx?c=AKR20180319157700004&did=1195m',
 'http://www.hankookilbo.com

In [160]:
len(listLink)

3990

In [75]:
# driver.close()
# driver.quit()

### 저장
- [dictionary to csv](https://stackoverflow.com/questions/8685809/python-writing-a-dictionary-to-a-csv-file-with-one-line-for-every-key-value?utm_medium=organic&utm_source=google_rich_qa&utm_campaign=google_rich_qa)
    - [writerow one line feed](https://stackoverflow.com/questions/3191528/csv-in-python-adding-an-extra-carriage-return?utm_medium=organic&utm_source=google_rich_qa&utm_campaign=google_rich_qa)

###### 저장 성공

In [163]:
import pandas as pd
from pandas import DataFrame
import re

In [164]:
DataFrame(listLink).to_csv("data/naverNewsSomePressLinks.csv", index= False, header=None)

### 읽기

In [127]:
mbLinkRead= pd.read_csv("data/mabinoticeLinks.csv", header=None)

In [128]:
mbLinkRead= list(mbLinkRead[0])

In [129]:
mbLinkRead[0:5]

['http://mabinogi.nexon.com/page/news/notice_view.asp?id=4886350',
 'http://mabinogi.nexon.com/page/news/notice_view.asp?id=4886336',
 'http://mabinogi.nexon.com/page/news/notice_view.asp?id=4886320',
 'http://mabinogi.nexon.com/page/news/notice_view.asp?id=4886304',
 'http://mabinogi.nexon.com/page/news/notice_view.asp?id=4886303']

In [130]:
len(mbLinkRead)

894

### 실행- 내용얻기_test

In [107]:
# data1= {'title': ['1','2'], 'date':['1','2'], 'body':['1','2'], 'writerP':['1','2']}
# dfEx= pd.DataFrame(data= data1)
# dfEx

In [139]:
driver.close()
driver.quit()

#### 링크에서 각 값 따오기-> DataFrame 만들기
conTitleText, conDateText, conBodyText, conWriterText

pd.DataFrame?

In [144]:
driver= startWeb()

In [145]:
# titleList= list([])
# dateList= list([])
# bodyList= list([])
# writerPList= list([])

# for i, link in enumerate(mbLinkRead):
#     #if (0== i):# | (1==i):
#     #print(link)
#     print(i)
#     movePageLink(link)

#     title, date, body, writerP= getContent()

#     titleList.append(title)
#     dateList.append(date)
#     bodyList.append(body)
#     writerPList.append(writerP)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [146]:
noticeData= { 
    'title':titleList,
    'date':dateList,
    'zbody':bodyList,
    'writerP':writerPList
}
dfNotice= pd.DataFrame(data= noticeData)

### 저장

In [147]:
dfNotice.to_csv('data/mbNoticeContent.csv', index= False, encoding= 'cp949')

In [1]:
# dfNotice['writerP'].value_counts()

In [2]:
import pandas as pd

## 읽기

In [3]:
df= pd.read_csv("data/mbNoticeContent.csv", encoding= 'cp949')

In [4]:
df.columns

Index(['date', 'title', 'writerP', 'zbody'], dtype='object')

#### 문자열 포함된 거 찾기

In [9]:
def findStr(inputStr):
    tradein= lambda x:inputStr in x
    return df[df.zbody.map(tradein)]

In [17]:
inputStr= '음식'
findStr(inputStr).to_csv('data/MBfind'+inputStr+'.csv', index= False, encoding= 'cp949')